In [1]:
import boto3
from dotenv import load_dotenv
import os
import re
import paramiko
import requests

Don't forget to **not use https** in the url


In [2]:
load_dotenv()
aws_access_key_id= os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")
region_name = os.getenv("region_name")

# Let's use Amazon S3
ec2 = boto3.client('ec2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

In [3]:
EC2_KWARGS = {
    "ImageId": "ami-07151644aeb34558a",  # ID des Amazon Linux 2 AMI
    "InstanceType": "t3.medium",  # Instanztyp t3.medium
    "MinCount": 1,  # Mindestanzahl von Instanzen
    "MaxCount": 1,  # Maximale Anzahl von Instanzen
    "KeyName": "Ec2-boto3",  # Name des Schlüsselpaars
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"],  # ID der Sicherheitsgruppe
    "IamInstanceProfile":{
        'Arn': "arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens"
    },
    "SubnetId":"subnet-0e19e3f6d404a4467",
    "BlockDeviceMappings": [
        {
            "DeviceName": "/dev/xvda",
            "Ebs": {
                "VolumeSize": 20,  # Größe des Root-Volumes in GB
                "DeleteOnTermination": True,
                "VolumeType": "gp3",
            },
        },
    ],
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
                sudo curl \
                    -L https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m) \
                    -o /usr/local/bin/docker-compose
                sleep 3    
                sudo chmod +x /usr/local/bin/docker-compose
             """,  # Skript, das beim Start der Instanz ausgeführt wird
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": "main-instance"},
            ],
        },
    ],
}

In [4]:
response = ec2.run_instances(**EC2_KWARGS)

# ID der gestarteten Instanz abrufen
instance_id = response['Instances'][0]['InstanceId']
print(f'Started instance {instance_id}')

Started instance i-01395d93a3ebd7241


In [5]:
response = ec2.associate_address(
    InstanceId=instance_id,
    PublicIp='3.66.239.204'
)
response

{'AssociationId': 'eipassoc-08857e80810baaa8d',
 'ResponseMetadata': {'RequestId': 'c0a635b5-dc19-4268-a838-f36869ec0123',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c0a635b5-dc19-4268-a838-f36869ec0123',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '295',
   'date': 'Mon, 03 Jul 2023 08:26:06 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

look up the public ip of the ec2 instance in the aws console

In [6]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('3.66.239.204', username='ec2-user', key_filename='Ec2-boto3.pem')

In [7]:
# Kopieren Sie die Docker Compose-Datei auf die Instanz
sftp = ssh.open_sftp()
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\.env', '/home/ec2-user/.env')
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\docker-compose.yml', '/home/ec2-user/docker-compose.yml')
sftp.close()

In [8]:
stdin, stdout, stderr = ssh.exec_command('sudo docker-compose up -d')

In [10]:
print(stdout)

<paramiko.ChannelFile from <paramiko.Channel 1 (closed) -> <paramiko.Transport at 0x25e11cf0 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>>>
